# Pre-trained MobileNet do transfer learning
下載 Keras-MobileNet model, using [Kaggle: Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/) dataset to do transfer learning.  
* [整理資料集](#%E6%95%B4%E7%90%86%E8%B3%87%E6%96%99%E9%9B%86)
  * 原本 training set 共有 25,000 筆，在此只取 6,000 筆資料來學習 (目前取用全部資料時，執行時間過長)
    * training images: 4000
    * validation images: 1000
    * test images: 1000
* [圖片預處理](#%E5%9C%96%E7%89%87%E9%A0%90%E8%99%95%E7%90%86)
  * ImageDataGenerator().flow_from_directory()
  * 需要 rescale 嗎? 目前沒做
* [Transfer-Learning](#Transfer-Learning)

## Reference
* [Transfer Learning with Keras！](https://ithelp.ithome.com.tw/articles/10190971)
* [Transfer learning from pre-trained models](https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751)
  * 如何將下載的資料整理成適當的目錄結構以供 ImageDataGenerator 使用。
* [keras_applications/mobilenet.py](https://github.com/keras-team/keras-applications/blob/master/keras_applications/mobilenet.py)
  * MobileNet model source code
* [Keras: MobileNet](https://keras.io/applications/#mobilenet)
  * Keras MobileNet class usage
* [Transfer Learning using Mobilenet and Keras](https://towardsdatascience.com/transfer-learning-using-mobilenet-and-keras-c75daf7ff299)
* [Keras 以 ResNet-50 預訓練模型建立狗與貓辨識程式](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)
* 基於Keras實現Kaggle2013–Dogs vs. Cats12500張貓狗影象的精準分類(https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/456610/)

In [1]:
from keras.preprocessing import image
from keras.applications.mobilenet import MobileNet, preprocess_input, decode_predictions
import numpy as np
import keras
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# Environment
!python --version
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
!nvidia-smi | grep Version | cut -c 3-

Python 3.7.3
TensorFlow: 1.13.1
Keras: 2.2.4
NVIDIA-SMI 415.27       Driver Version: 415.27       CUDA Version: 10.0     |


In [3]:
# Workaround Issue: "Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

### 整理資料集
* 從 [Kaggle: Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats/) 下載資料
* 建立好目錄結構  
  * train (各2,000張) 
    * cats  
    * dogs  
  * valid (各500張)    
    * cats
    * dogs  
  * test (各500張)    
    * cats
    * dogs  

In [4]:
import os, shutil
# import os, shutil, glob

sour_dir = '/home/ryanyao/docker_mount/data/dogs_vs_cats'
dest_dir = '/home/ryanyao/docker_mount/data/dogs_vs_cats_dir'
# 目前 test 目錄未使用
step_set = ('train', 'valid', 'test')
class_set = ('cats', 'dogs')
class_fname_set = ('cat', 'dog')

In [5]:
if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

# Create directories
for i_set in step_set:
    set_dir = os.path.join(dest_dir, i_set)
    if not os.path.exists(set_dir):
        os.mkdir(set_dir)
    for i_class in class_set:
        class_dir = os.path.join(set_dir, i_class)
        if not os.path.exists(class_dir):
            os.mkdir(class_dir)

# # Copy all cat/dog images to train_dir
# for i_idx, i_class in enumerate(['cat', 'dog']):
#     sour_path = (os.path.join(sour_dir, step_set[0], i_class+"*.jpg"))
#     fnames = glob.glob(sour_path)
#     for fname in fnames:
#         dst = os.path.join(dest_dir, step_set[0], class_set[i_idx], os.path.basename(fname))
#         shutil.copyfile(fname, dst)                                 

# Copy 3000 cat/dog images to train_dir
for i_idx, i_class in enumerate(class_fname_set):
    fname_pattern = '%s.{}.jpg' % i_class
    fnames = [fname_pattern.format(i) for i in range(0, 3000)]
    for fname in fnames:
        src = os.path.join(sour_dir, step_set[0], fname)
        det = os.path.join(dest_dir, step_set[0], class_set[i_idx], fname)
        shutil.copyfile(src, det)

In [6]:
# Move 500 cat/dog images to valid_dir. range(1000, 1500)
# Move 500 cat/dog images to test_dir. range(1500, 2000)
for i_step in step_set[1:3]:
    for i_idx, i_class in enumerate(class_set):
        fname_pattern = '%s.{}.jpg' % class_fname_set[i_idx]
        if i_step == 'valid':
            fnames = [fname_pattern.format(i) for i in range(1000, 1500)]
        else:
            fnames = [fname_pattern.format(i) for i in range(1500, 2000)]
        # print(fnames)
        for fname in fnames:
            src = os.path.join(dest_dir, step_set[0], i_class, fname)
            det = os.path.join(dest_dir, i_step, i_class, fname)
            shutil.move(src, det)

In [7]:
# Sanity checks
print('total training cat images:', len(os.listdir(os.path.join(dest_dir, step_set[0], class_set[0]))))
print('total training dog images:', len(os.listdir(os.path.join(dest_dir, step_set[0], class_set[1]))))
print('total validation cat images:', len(os.listdir(os.path.join(dest_dir, step_set[1], class_set[0]))))
print('total validation dog images:', len(os.listdir(os.path.join(dest_dir, step_set[1], class_set[1]))))
print('total test cat images:', len(os.listdir(os.path.join(dest_dir, step_set[2], class_set[0]))))
print('total test dog images:', len(os.listdir(os.path.join(dest_dir, step_set[2], class_set[1]))))

total training cat images: 2000
total training dog images: 2000
total validation cat images: 500
total validation dog images: 500
total test cat images: 500
total test dog images: 500


In [8]:
import numpy as np
import keras
from keras.layers.core import Dense, Flatten
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

## 圖片預處理


In [9]:
img_base = '/home/ryanyao/docker_mount/data/dogs_vs_cats_dir/'
train_path = os.path.join(img_base,'train')
valid_path = os.path.join(img_base,'valid')
test_path = os.path.join(img_base,'test')
print(train_path, valid_path, test_path, sep='\n')

/home/ryanyao/docker_mount/data/dogs_vs_cats_dir/train
/home/ryanyao/docker_mount/data/dogs_vs_cats_dir/valid
/home/ryanyao/docker_mount/data/dogs_vs_cats_dir/test


In [10]:
batch_size = 8
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224,224), classes=['dogs', 'cats'], batch_size=batch_size)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224,224), classes=['dogs', 'cats'], batch_size=batch_size)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224,224), classes=['dogs', 'cats'], batch_size=batch_size)

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
# 檢視 image input size
print(train_batches.image_shape)

(224, 224, 3)


### Transfer Learning
* 下載 Keras NobileNet model 進行 transfer learning，並計算其 test dataset 的正確率

In [12]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = MobileNet(weights='imagenet', include_top=False) 

x = base_model.output
# we add dense layers so that the model can learn more complex functions and classify for better results.
x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x) 
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x) # final layer with softmax activation

Instructions for updating:
Colocations handled automatically by placer.


/home/ryanyao/anaconda3/lib/python3.7/site-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


In [13]:
model = Model(inputs=base_model.input, outputs=preds)

In [14]:
# # Check the model architecture
# for i,layer in enumerate(model.layers):
#     print(i, layer.name, layer.trainable)

In [15]:
# We will only train the last few dense layers.
# if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [16]:
# 輸出整個網路結構
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_batches.n // train_batches.batch_size
step_size_valid = valid_batches.n // valid_batches.batch_size
history = model.fit_generator(generator=train_batches, 
                    steps_per_epoch=step_size_train, 
                    validation_data=valid_batches,
                    validation_steps=step_size_valid,
                    epochs=10,
                    verbose=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 - 21s - loss: 0.3044 - acc: 0.8888 - val_loss: 0.9725 - val_acc: 0.5060
Epoch 2/10
 - 17s - loss: 0.1974 - acc: 0.9295 - val_loss: 0.6346 - val_acc: 0.6420
Epoch 3/10
 - 17s - loss: 0.1421 - acc: 0.9515 - val_loss: 0.7035 - val_acc: 0.5660
Epoch 4/10
 - 17s - loss: 0.1006 - acc: 0.9647 - val_loss: 0.6997 - val_acc: 0.5430
Epoch 5/10
 - 17s - loss: 0.1116 - acc: 0.9607 - val_loss: 2.4261 - val_acc: 0.5000
Epoch 6/10
 - 17s - loss: 0.0846 - acc: 0.9700 - val_loss: 1.4644 - val_acc: 0.5010
Epoch 7/10
 - 17s - loss: 0.0578 - acc: 0.9778 - val_loss: 0.8009 - val_acc: 0.5990
Epoch 8/10
 - 17s - loss: 0.0687 - acc: 0.9755 - val_loss: 0.8377 - val_acc: 0.4670
Epoch 9/10
 - 17s - loss: 0.0486 - acc: 0.9840 - val_loss: 2.0646 - val_acc: 0.5530
Epoch 10/10
 - 17s - loss: 0.0420 - acc: 0.9860 - val_loss: 0.6224 - val_acc: 0.6440


In [18]:
step_size_test = test_batches.n // test_batches.batch_size
model.evaluate_generator(generator=test_batches, steps=step_size_test, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=2)

[0.6372752715349197, 0.631]